## Face Mask Detection with WebCam: Using OpenCV Haar Cascade

Here we will be using OpenCV's Haar Cascade for detecting the face and test wether the face has a mask or not to and to show the status live on the screenc with a webcam.<br><br>
Here are the steps we need to follow:<pre>
    1. First, provide the xml file for the Haar Cascade, named as <b>"haarcascade_frontalface_default.xml"</b>
    2. Then, load the model we built earlier with MobileNet and compile it.
    3. Create a function that will only crop the area where the face at and use that image frame for predicting the results via the model we created.
    4. We also create a function where we set the parameters of the Haar Cascade classifier to detect the face and show results based on the conditions we ought to give.
    5. Lastly, we set the video capture, window size, and terminating points before we call our functions to do the tasks.
</pre>

### Import Necessary Libraries

In [1]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

### Initiate Haar Cascade XML file with OpenCV

In [2]:
face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

### Load Model and Compile

In [3]:
mask_detection = tf.keras.models.load_model("mask_detection_mb.h5", compile=False)

mask_detection.compile(
    optimizer='SGD', 
    loss='binary_crossentropy', 
    metrics=['accuracy']
)

### Function for Face Frame and Prediction

In [4]:
def predict(image):
    face_frame = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    face_frame = cv2.resize(face_frame, (224, 224))
    face_frame = img_to_array(face_frame)
    face_frame = np.expand_dims(face_frame, axis=0)
    face_frame =  preprocess_input(face_frame)
    
    # predict the selected face frame for mask
    prediction = mask_detection.predict(face_frame)
    
    return prediction[0][0]

### Function for Haar Cascade Face Detection

In [5]:
text_mask = "Mask On"
text_no_mask = "Mask Off"
font = cv2.FONT_HERSHEY_SIMPLEX
scale = 0.8

def detect_and_draw(frame):
    # For grayscale image
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Initialize Haar Cascade
    faces = face_classifier.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)
    for (x, y, w, h) in faces:
        roi_color = frame[y:y+h, x:x+w]
        mask = predict(roi_color)
        color = (0, 255, 0) if mask > 0.5 else (0, 0, 255)
        label = text_mask if mask > 0.5 else text_no_mask
        
        # Frame size for the rectangular detection frame
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        # Output text after the predictions made
        cv2.putText(frame, text=label, org=(x+5, y-10), fontFace=font, 
                    fontScale=scale, color=color, thickness=2)

    return frame

### Use the Webcam for live Testing

In [6]:
# Initialize the video capture
video_cap = cv2.VideoCapture(0)

# Set the desired window size
window_width = 640
window_height = 480
window_name = "Mask Detection Webcam"

cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name, window_width, window_height)

while True:
    ret, frame = video_cap.read()
    if not ret:
        break

    # Perform face detection and annotation
    annotated_frame = detect_and_draw(frame)

    # Resize and display the frame
    resized_frame = cv2.resize(annotated_frame, (window_width, window_height))
    cv2.imshow(window_name, resized_frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release resources
video_cap.release()
cv2.destroyAllWindows()